In [1]:
from lxml import html
import requests
import urllib

In [2]:
# wikihow has a rather streamline API
url = "https://www.wikihow.com/Play-Gin-Rummy"
url = "https://www.wikihow.com/Overcome-an-Avoidant-Personality-Disorder"

# there also is a RU wikihow website
url = "https://ru.wikihow.com/%D0%BD%D0%B0%D1%83%D1%87%D0%B8%D1%82%D1%8C%D1%81%D1%8F-%D0%B1%D1%8B%D1%81%D1%82%D1%80%D0%B5%D0%B5-%D0%B1%D0%B5%D0%B3%D0%B0%D1%82%D1%8C"

# we can access random pages using a special enpoint
wikihow_url = "https://www.wikihow.com"
random_page_url = "Special:Randomizer"

# similar endpoint exists for RU version
random_page_url_ru = "%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:Randomizer"
wikihow_url_ru = "https://ru.wikihow.com"

In [3]:
# with requests.get(url=f"{wikihow_url_ru}/{random_page_url_ru}") as response:
# with requests.get(url="https://ru.wikihow.com/%D0%BF%D0%BE%D1%85%D1%83%D0%B4%D0%B5%D1%82%D1%8C-%D0%BC%D0%B5%D0%B4%D0%BB%D0%B5%D0%BD%D0%BD%D0%BE") as response:
#     tree = html.fromstring(response.text)
#     print(response.url)

# https://www.wikihow.com/Reset-Your-Life
with requests.get(url=f"{wikihow_url}/{random_page_url}") as response:
    tree = html.fromstring(response.text)
    # URL is generated dynamically and is unknown in prior,
    # but can be accessed via .url property of response object
    print(response.url)

https://www.wikihow.com/Clean-a-Mother-Dog-After-She-Gives-Birth


In [4]:
# xpath cheatsheet: https://devhints.io/xpath#class-check

# no idea how to simplify these further,
# but these queries seem to work nicely for both EN and RU versions
# of the website
query = "|".join(
    (
        "//div[@class='step']/text()",
        "//div[@class='step']/b/text()",
        "//div[@class='step']/ul//li/text()",
    )
)

num_words = sum(len(x.split()) for x in
    filter(
        lambda x: x and len(x) > 1,
        map(
            lambda x: x.strip() if x else None,
            tree.xpath(query),
        ),
    )
)

print(f"words: {num_words}")

for i, e in enumerate(
    filter(
        lambda x: x and len(x) > 1,
        map(
            lambda x: x.strip() if x else None,
            tree.xpath(query),
        ),
    ),
):
    print(i, "-", e.strip())


words: 775
0 - Remove all soiled items after the birth.
1 - After the mother dog is finished giving birth, you should remove all the soiled newspaper, blankets, or other material. This material will contain blood, fluid, and tissue from the birth.
2 - Removing the soiled items promptly reduces the risk of contamination and infection.
3 - Fix the mother and her litter a clean, soft bed.
4 - After you remove the soiled material in the whelping box, replace it with clean blankets. You may consider placing newspaper down, especially in the first few days because the mother may not leave the puppies, even to go to the bathroom.
5 - Make sure the bedding is appropriate for the puppies, as well. It should not have any loose threads or large loops that could wrap around limbs, no holes that the puppies could become trapped under, and nothing so big or plush that it could smother a puppy.
6 - Replace the bedding daily.
7 - During the first few weeks, the bedding in the whelping box should be re

In [5]:
# we can go futher and instead of generating pages at random,
# extract pages which wiki itself considers similar (EN version
# also provides "trending" articles, which can also be useful)
query = "|".join(
    (
        "//a[@class='related-wh']/@href",
    )
)

for i, e in enumerate(
    filter(
        lambda x: x and len(x) > 1,
        map(
            lambda x: x.strip() if x else None,
            tree.xpath(query),
        ),
    ),
):
    print(i, "-", e.strip())

0 - /Determine-the-Sex-of-Puppies
1 - /Calm-a-Male-Dog-when-a-Female-Is-in-Heat
2 - /Know-if-Your-Female-Dog-Is-Ready-to-Breed
3 - /Encourage-Dogs-to-Mate-Naturally
4 - /Feed-and-Hydrate-a-Mother-Dog
5 - /Revive-a-Puppy
6 - /Help-a-Dog-in-Heat
7 - /Deal-with-Puppy-Deaths-During-Birth
8 - /Get-Dogs-to-Mate
9 - /Know-if-Your-Male-Dog-Is-Ready-to-Breed
10 - /Tell-if-Your-Dog-Is-in-Heat
11 - /Know-if-a-Male-Dog-Is-Fertile
12 - /Breed-French-Bulldogs
13 - /Deal-with-Common-Mating-Problems-in-Dogs
14 - /Determine-the-Sex-of-Puppies
15 - /Calm-a-Male-Dog-when-a-Female-Is-in-Heat
16 - /Know-if-Your-Female-Dog-Is-Ready-to-Breed
17 - /Encourage-Dogs-to-Mate-Naturally


In [ ]:
# url can be decoded into human-readable string with urllib:
# also see (bash aliases!): https://unix.stackexchange.com/questions/159253/decoding-url-encoding-percent-encoding
url = "https://ru.wikihow.com/%D0%BD%D0%B0%D1%83%D1%87%D0%B8%D1%82%D1%8C%D1%81%D1%8F-%D0%B1%D1%8B%D1%81%D1%82%D1%80%D0%B5%D0%B5-%D0%B1%D0%B5%D0%B3%D0%B0%D1%82%D1%8C"
urllib.parse.unquote(urllib.parse.urlparse(url).path)

In [ ]:
# in such a manner, we can traverse the service and extract similar articles,
# and sometimes pick pages from randomly generated ones.

# I suppose that in sake of reproducibility, we would first extract a reasonable
# number of pages and 